<a href="https://colab.research.google.com/github/Konstantan99/to-determine-the-gender-of-a-person/blob/new_proba/to_determine_the_gender_of_a_person.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Предлагается поработать с банковскими данными и решить несколько исследовательских задач: 
определить пол клиента по его финансовым тратам.


*   определить пол клиента по его финансовым тратам.


Основные данные представляют из себя историю банковских транзакций, а также демографическую информацию по некоторой выборке клиентов (данные обезличены и специальным образом искажены).


In [ ]:
import pandas as pd
import numpy as np
import sklearn.model_selection

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

import xgboost
#from sklearn.model_selection import cross_validation
from sklearn.metrics import accuracy_score

from sklearn.ensemble import GradientBoostingClassifier


import plotly.graph_objects as go

In [ ]:
#ввод исходников с GitHub

url_customers_gender_train = 'https://raw.githubusercontent.com/Konstantan99/to-determine-the-gender-of-a-person/main/customers_gender_train.csv' 
customers_gender_train = pd.read_csv(url_customers_gender_train, delimiter=',')

url_tr_mcc_codes = 'https://raw.githubusercontent.com/Konstantan99/to-determine-the-gender-of-a-person/main/tr_mcc_codes.csv' 
tr_mcc_codes = pd.read_csv(url_tr_mcc_codes)
del tr_mcc_codes['Unnamed: 2']
del tr_mcc_codes['Unnamed: 3']
del tr_mcc_codes['Unnamed: 4']
del tr_mcc_codes['Unnamed: 5']

url_transactions = 'https://raw.githubusercontent.com/Konstantan99/to-determine-the-gender-of-a-person/main/transactions.csv' 
transactions = pd.read_csv(url_transactions)

print(
    '''
Размеры выборок, (объекты, признаки).
- customers_gender_train: {}
- tr_mcc_codes: {}
- transactions: {}
    '''.format(
        customers_gender_train.shape,  
        tr_mcc_codes.shape,
        transactions.shape
    )
)


Размеры выборок, (объекты, признаки).
- customers_gender_train: (12000, 2)
- tr_mcc_codes: (184, 2)
- transactions: (499999, 6)
    


# Описание таблиц

**ОПИСАНИЕ ИСХОДНЫХ ДАННЫХ**


---


**Таблица transactions.csv**

***Описание:*** Таблица содержит историю транзакций клиентов банка за один год и три месяца.

***Описание полей:***
*   customer_id — идентификатор клиента;
*   tr_datetime — день и время совершения транзакции (дни нумеруются с начала данных);
*   mcc_code — mcc-код транзакции;
*   tr_type — тип транзакции;
*   amount — сумма транзакции в условных единицах со знаком; + — начисление средств клиенту (приходная транзакция), - — списание средств (расходная транзакция);
*   term_id — идентификатор терминала;


---


***Таблица customers_gender_train.csv***

***Описание:*** Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол необходимо предсказать в задаче A.

Описание полей:
*   customer_id — идентификатор клиента;
*   gender — пол клиента; 0 — женский, 1 — мужской;



---


**Таблица tr_mcc_codes.csv**

***Описание:*** Данная таблица содержит описание mcc-кодов транзакций.

Формат данных

mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000

Описание полей:
*   mcc_code – mcc-код транзакции;
*   mcc_description — описание mcc-кода транзакции.


---


**Таблица tr_types.csv**
***Описание:*** Данная таблица содержит описание типов транзакций.

Описание полей:
*   tr_type – тип транзакции;
*   tr_description — описание типа транзакции.

In [ ]:
customers_gender_train.head(5)

,customer_id,gender
0,75562265,0
1,10928546,1
2,69348468,1
3,84816985,1
4,61009479,0


In [ ]:
total = transactions.isnull().sum().sort_values(ascending=False) #определяем количество пропусков
percent = (transactions.isnull().sum()/transactions.isnull().count()).sort_values(ascending=False) #определяем количество пропусков
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data

,Total,Percent
term_id,211067,0.422135
amount,0,0.000000
tr_type,0,0.000000
mcc_code,0,0.000000
tr_datetime,0,0.000000
customer_id,0,0.000000


In [ ]:
transactions.head(5)

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN
1,39026145,1 10:19:29,6011,7010,56147.89,NaN
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN
4,39026145,2 15:33:42,5499,1010,-920.83,NaN


In [ ]:
tr_mcc_codes.head(5)

,mcc_code,mcc_description
0,742,Ветеринарные услуги
1,1711,Генеральные подрядчики по вентиляции
2,1731,Подрядчики по электричеству
3,1799,Подрядчики
4,2741,Разнообразные издательства/печатное дело


# Определение времени по исходным данным

Рассмотрим транзакции из флористики 

In [ ]:
transactions_floristic = transactions[transactions['mcc_code'] == 5992]
time_df = transactions_floristic['tr_datetime'].str.split(' ', expand=True) #разделяем временной столбец на день и время
time_df.columns=['number_day','time']
transactions_floristic = pd.concat([transactions_floristic, time_df], axis=1)
transactions_floristic['number_day'] = pd.to_numeric(transactions_floristic['number_day'])

df_for_plot = pd.DataFrame(transactions_floristic['number_day'].value_counts()) #определяем сколько прошло транзакций в каждый по счету день 
df_for_plot = df_for_plot.reset_index()
df_for_plot.columns = ['day','count_day']
df_for_plot['percent_transactions'] = df_for_plot['count_day'] / df_for_plot['count_day'].sum() * 100 #определяем относительное количество покупок

df_for_plot = df_for_plot.sort_values('day', ascending=True) #сортируем день по возрастанию

,day,count_day,percent_transactions
297,0,1,0.198413
182,1,1,0.198413
181,3,1,0.198413
180,5,1,0.198413
179,7,1,0.198413


In [ ]:
# Создание фигуры точечного графика
fig = go.Figure(data=go.Scatter(
    x=df_for_plot['day'], y=df_for_plot['percent_transactions']
))
# Добавление подписей для графика
fig.update_layout(
    title_font_size=20,
    xaxis_title="День по счету",
    yaxis_title="Количество транзакций",  
)
# Отображение графика
fig.show()

NameError: ignored

# Подготовка данных для составления модели

Оставляем те строки, по которым мы знаем пол из таблицы `customers_gender_train`. 

Для Y значений также оставляем только те, по которым мы имеем транзакции

In [ ]:
needed_transaction = transactions[transactions['customer_id'].isin(customers_gender_train['customer_id'])]
Y = customers_gender_train[customers_gender_train['customer_id'].isin(needed_transaction['customer_id'])]

Разделим столбец `tr_datetime` из вида `"1 10:20:56"`	на день и время

In [ ]:
time_df = needed_transaction['tr_datetime'].str.split(' ', expand=True) #разделяем временной столбец на день и время
time_df.columns=['number_day','time']
needed_transaction = pd.concat([needed_transaction, time_df], axis=1)
del needed_transaction['tr_datetime']

#преобразуем данные в нужные форматы
needed_transaction['number_day'] = pd.to_numeric(transactions_floristic['number_day'])
needed_transaction['time'] = pd.to_datetime(needed_transaction['time'], format='%H:%M:%S')
needed_transaction

## Количество транзакций клиента по каждому типу транзакций

Перестроим датафрейм: с группируем данные для каждого покупателя, и посмотрим сколько покупок совершенно в каждом типе транзакций (mcc_code)

In [ ]:
X = needed_transaction.groupby('customer_id') \
                    .apply(lambda x: x[['mcc_code']].unstack().value_counts()) \
                    .unstack() \
                    .fillna(0)

Сортируем данные по `customer_id`, чтоб правильно сопоставить признаки с искомым признаком (пол клиента) 

In [ ]:
X = X.sort_values(by='customer_id')
Y = Y.sort_values(by='customer_id')
del Y['customer_id'] #удаляем столбец customer_id за ненадобностью

#проверить правильно ли сортируются данные

# Признаки по тратам и поступлениям

Разделим все транзакции на траты и поступления

In [ ]:
income_transactions = needed_transaction[needed_transaction['amount'] > 0]
expenses_transactions = needed_transaction[needed_transaction['amount'] < 0]

Для каждого вида по каждому клиенту определим:


*   Максимальное значение (для трат - минимальное)
*   Среднее значение
*   Медианное значение
*   Стандартное отклонение
*   Количество транзакций
*   Общая сумма





In [ ]:
income_df = income_transactions.groupby('customer_id').agg({'amount': ['count', 'sum', 'max', 'mean', 'median', 'std']})
income_df.columns=['count_income','sum_income','max_income','mean_income','median_income','std_income']

In [ ]:
expense_df = expenses_transactions.groupby('customer_id').agg({'amount': ['count', 'sum', 'min', 'mean', 'median', 'std']})
expense_df.columns=['count_expense', 'sum_expense','max_expense','mean_expense','median_expense','std_expense']

In [ ]:
#добавить общие?

Присоеденим полученные признаки к датафрейму `Х`:

In [ ]:
X = X.merge(income_df, on='customer_id', how='left')
X = X.merge(expense_df, on='customer_id', how='left')
X = X.fillna(0) #заполним пустые ячейки (клиенты, у которых остутсвуют доходы или расходы, или стандартное отклонение равно нулю)

# Разбиение данных на train и test. Настройка модели

In [ ]:
# split data into train and test sets
seed = 7
test_size = 0.15
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)

In [ ]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import logging as lgr

train = xgboost.DMatrix(X_train, Y_train)

def score(params):
  seed = int(np.random.rand()*100000)
  params['max_depth'] = int(params['max_depth'])
  lgr.info('seed = %i' % seed)
  lgr. info ("Training with params: ")
  lgr.info(params)
  cv_res = xgboost.cv(params, train, early_stopping_rounds=100, maximize=True, 
                  num_boost_round=10000, nfold=5, seed = seed) 
  score = cv_res['test-auc-mean'].max() 
  lgr.info("Score = %f" % score)
  lgr.info('best rounds = %i' % cv_res[cv_res['test-auc-mean'] == cv_res['test-auc-mean'].max()].index[0])
  return {'loss': -score, 'status': STATUS_OK}

Функция оценки возвращает результат со знаком минус — это сделано специально, т.к. hyperopt минимизирует функционал, а нам надо максимизировать точность.

In [ ]:
space = {
    'eta' : hp.quniform('eta', 0.001, 0.1, 0.001),
    'max_depth' : hp.quniform('max_depth', 3, 15, 1),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 30, 1),
    'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
    'gamma' : hp.quniform('gamma', 0.1, 2, 0.05),
    'alpha': hp.quniform('alpha', 0.001, 2, 0.05),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.01, 1, 0.01),
    'eval_metric': 'auc',
    'objective': 'binary:logistic',
    'booster': 'gbtree',
    'nthread' : 11,
    'silent' : 1
    }
trials = Trials() 
best = fmin(fn=score,
            space=space,
            algo=tpe.suggest,
            trials=trials,
            max_evals=500)

100%|██████████| 5/5 [00:55<00:00, 11.15s/it, best loss: -0.8635216]


In [ ]:
print('''best parameters:
    alpha={}, 
    colsample_bytree={}, 
    eta={},
    gamma={},
    max_depth={}, 
    min_child_weight={}, 
    subsample={}
    '''.format(
        best['alpha'],
        best['colsample_bytree'],
        best['eta'],
        best['gamma'],
        round(best['max_depth']),
        best['min_child_weight'],
        best['subsample']
    ))

best parameters:
    alpha=0.15000000000000002, 
    colsample_bytree=0.43, 
    eta=0.067,
    gamma=0.4,
    max_depth=5, 
    min_child_weight=2.0, 
    subsample=0.6000000000000001
    


In [ ]:
def df_results(hp_results):
    """
    Отображаем результаты hyperopt в формате DataFrame 

    :hp_results: результаты hyperop
    :return: pandas DataFrame
    """ 

    results = pd.DataFrame(hp_results)
    results['loss'] = results['loss'] * (-1)
    results.drop(labels=['status'], axis=1, inplace=True)
    return results

results = df_results(trials.results)
results

,loss
0,0.849452
1,0.849826
2,0.838235
3,0.863522
4,0.836693


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=results.index, y=results['loss']
))
# Добавление подписей для графика
fig.update_layout(
    title_font_size=20,
    xaxis_title="День по счету",
    yaxis_title="Количество транзакций",  
)
# Отображение графика
fig.show()

Настроим модель XGBClassifier с подобранными параметрами:


    alpha=0.9500000000000001, 
    colsample_bytree=0.08, 
    eta=0.022,
    gamma=1.4500000000000002,
    max_depth=15, 
    min_child_weight=26.0, 
    subsample=0.9500000000000001


In [ ]:
# fit model no training data
model = xgboost.XGBClassifier(
    alpha=0.15000000000000002, 
    colsample_bytree=0.43, 
    eta=0.067,
    gamma=0.4,
    max_depth=5, 
    min_child_weight=2.0, 
    subsample=0.6000000000000001
)
model.fit(X_train, Y_train)

# make predictions for test data
y_pred = model.predict(X_test)
print(y_pred)
print()
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 0 0 1 1 1 0 1 0 1 0 0
 0 1 0 0 0 1 1 1 1 0 0 1 0 1 0 0 1 0 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 0
 1 0 0 1 1 1 1 0 0 1 0 1 0 1 1 1 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 0 1 1 1 0 1
 1]

Accuracy: 83.04%
